In [2]:
from neural_net import NeuralNet
from neural_net import LayerType
from neural_net import LayerDesc
from neural_net import ActivationType

ImportError: /home/tyler/projects/UNet/.venv/lib/python3.12/site-packages/neural_net.cpython-312-x86_64-linux-gnu.so: undefined symbol: _ZN5Layer16forward_upsampleEPfibbb

In [2]:
import os
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt

def load_dataset(root_dir):
    """
    Recursively parse the LGG MRI dataset.
    Returns a list of dictionaries:
    [
      {
        "image_path": "...",
        "mask_path": "...",
        "image": np.ndarray,
        "mask": np.ndarray
      },
      ...
    ]
    """
    data = []

    # Walk through all subdirectories
    for subdir, dirs, files in os.walk(root_dir):
        # Only consider tif files
        tifs = [f for f in files if f.endswith(".tif")]

        # Create mapping: index -> file
        image_files = {f.replace("_mask", ""): f for f in tifs if "_mask" not in f}
        mask_files  = {f.replace("_mask", ""): f for f in tifs if "_mask" in f}

        # Pair image + mask files
        for key in sorted(image_files.keys()):
            if key in mask_files:
                img_path = os.path.join(subdir, image_files[key])
                mask_path = os.path.join(subdir, mask_files[key])

                # Load arrays
                img = np.array(Image.open(img_path))
                mask = np.array(Image.open(mask_path))

                data.append({
                    "image_path": img_path,
                    "mask_path": mask_path,
                    "image": img,
                    "mask": mask
                })

    return data


if __name__ == "__main__":
    root = "/home/tyler/projects/UNet/data/lgg-mri-segmentation/kaggle_3m"
    dataset = load_dataset(root)

    print(f"Loaded {len(dataset)} image/mask pairs.\n")

    # Example: print one sample
    sample = dataset[0]
    print("Image:", sample["image_path"])
    print("Mask:", sample["mask_path"])
    print("Image shape:", sample["image"].shape)
    print("Mask shape:", sample["mask"].shape)

    # ---- Split into tumor / non-tumor ----
    tumor = []
    no_tumor = []
    
    for item in dataset:
        if np.any(item["mask"] > 0):
            tumor.append(item)
        else:
            no_tumor.append(item)
    
    print("Tumor images:", len(tumor))
    print("No-tumor images:", len(no_tumor))
    
    images = np.stack([
        np.transpose(item["image"], (2, 0, 1))  # (H,W,C) → (C,H,W)
        for item in dataset
    ]).astype(np.float32)                        # final shape [N, 3, 256, 256]

    masks = np.stack([
        item["mask"]                             # (H,W)
        for item in dataset
    ]).astype(np.float32)                          # final shape [N, 256, 256]
    images = np.ascontiguousarray(images)
    masks = np.ascontiguousarray(masks)
    N, C, H, W = images.shape
    shape = [float(N), float(C), float(H), float(W)]



Loaded 3929 image/mask pairs.

Image: /home/tyler/projects/UNet/data/lgg-mri-segmentation/kaggle_3m/TCGA_DU_8164_19970111/TCGA_DU_8164_19970111_1.tif
Mask: /home/tyler/projects/UNet/data/lgg-mri-segmentation/kaggle_3m/TCGA_DU_8164_19970111/TCGA_DU_8164_19970111_1_mask.tif
Image shape: (256, 256, 3)
Mask shape: (256, 256)
Tumor images: 1373
No-tumor images: 2556


In [ ]:
# assume strides and sizes tile image correctly without padding
max_pool_stride = 2
max_pool_size = 2
H1 = 256
H2 = H1 / max_pool_stride
H3 = H2 / max_pool_stride
H4 = H3 / max_pool_stride

W1 = 256
W2 = W1 / max_pool_stride
W3 = W2 / max_pool_stride
W4 = W3 / max_pool_stride
in_channels = 3
F1 = 64
F2 = 2 * F1
F3 = 2 * F2
F4 = 2 * F3

# keep encoder and decoder even
upsample_factor = max_pool_stride

UNET = NeuralNet()  # instantiate the C++ class
UNET.create(
    [
        # parameters are in_height, in_width, in_channels, out_channels, kernel size, padding, and stride
        LayerDesc(LayerType.CONV_LAYER, [H1, W1, in_channels, F1, 3, 1, 1], []),  # 0
        # parameters are in_channels, in_height, in_width, size and stride
        LayerDesc(
            LayerType.MAX_POOL_LAYER, [F1, H1, W1, max_pool_size, max_pool_stride], [0]
        ),  # 1
        LayerDesc(LayerType.CONV_LAYER, [H2, W2, F1, F2, 3, 1, 1], [1]),  # 2
        LayerDesc(
            LayerType.MAX_POOL_LAYER, [F2, H2, W2, max_pool_size, max_pool_stride], [2]
        ),  # 3
        LayerDesc(LayerType.CONV_LAYER, [H3, W3, F2, F3, 3, 1, 1], [3]),  # 4
        LayerDesc(
            LayerType.MAX_POOL_LAYER, [F3, H3, W3, max_pool_size, max_pool_stride], [4]
        ),  # 5
        LayerDesc(LayerType.CONV_LAYER, [H4, W4, F3, F4, 3, 1, 1], [5]),  # 6
        # parameters are h_in, w_in, c_in, c_out, stride/upscale
        LayerDesc(
            LayerType.UPSAMPLING_LAYER, [H4, W4, F4, F3, upsample_factor], [6]
        ),  # 7
        # parameters are fully determined by parents. First parent is skip connection, second is convolution below
        LayerDesc(LayerType.ATTENTION_LAYER, [], [4, 6]),  # 8
        # parameters are fully determined by parents. First parent is from skip/attention, second is from convolution below
        LayerDesc(LayerType.CONCAT_LAYER, [], [8, 7]),  # 9
        LayerDesc(LayerType.CONV_LAYER, [H3, W3, F4, F3, 3, 1, 1], [9]),  # 10
        LayerDesc(
            LayerType.UPSAMPLING_LAYER, [H3, W3, F2, F2, upsample_factor], [10]
        ),  # 11
        LayerDesc(LayerType.ATTENTION_LAYER, [], [2, 10]),  # 12
        LayerDesc(LayerType.CONCAT_LAYER, [], [12, 11]),  # 13
        LayerDesc(LayerType.CONV_LAYER, [H2, W2, F3, F1, 3, 1, 1], [13]),  # 14
        LayerDesc(
            LayerType.UPSAMPLING_LAYER, [H2, W2, F1, F1, upsample_factor], [14]
        ),  # 15
        LayerDesc(LayerType.ATTENTION_LAYER, [], [0, 14]),  # 16
        LayerDesc(LayerType.CONCAT_LAYER, [], [16, 15]),  # 17
        LayerDesc(LayerType.CONV_LAYER, [H1, W1, F2, 1, 3, 1, 1], [17], ActivationType.SIGMOID),  # 18
        
    ]
)


learning_rate = 0.001
epochs = 10
batch_size = 16
UNET.train(images, shape, masks, learning_rate, epochs, batch_size)

beggining epoch 0/10
batch: 0
batch: 1
batch: 2
batch: 3
batch: 4
batch: 5
batch: 6
batch: 7
batch: 8
batch: 9
batch: 10
batch: 11
batch: 12
batch: 13
batch: 14
batch: 15
batch: 16
batch: 17
batch: 18
batch: 19
batch: 20
batch: 21
batch: 22
batch: 23
batch: 24
batch: 25
batch: 26
batch: 27
batch: 28
batch: 29
batch: 30
batch: 31
batch: 32
batch: 33
batch: 34
batch: 35
batch: 36
batch: 37
batch: 38
batch: 39
batch: 40
batch: 41
batch: 42
batch: 43
batch: 44
batch: 45
batch: 46
batch: 47
batch: 48
batch: 49
batch: 50
batch: 51
batch: 52
batch: 53
batch: 54
batch: 55
batch: 56
batch: 57
batch: 58
batch: 59
batch: 60
batch: 61
batch: 62
batch: 63
batch: 64
batch: 65
batch: 66
batch: 67
batch: 68
batch: 69
batch: 70
batch: 71
batch: 72
batch: 73
batch: 74
batch: 75
batch: 76
batch: 77
batch: 78
batch: 79
batch: 80
batch: 81
batch: 82
batch: 83
batch: 84
batch: 85
batch: 86
batch: 87
batch: 88
batch: 89
batch: 90
batch: 91
batch: 92
batch: 93
batch: 94
batch: 95
batch: 96
batch: 97
batch: 98

In [ ]:
print(dataset[2]["image"].shape)
print(dataset[2]["image"][40][30][1])
print(len(dataset)/16)


(256, 256, 3)
3
245.5625
